### Import neccessary library

In [1]:
# !pip install tqdm

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from tqdm import tqdm
import requests


opts = webdriver.ChromeOptions()
opts.headless = True
browser = webdriver.Chrome(options=opts)
browser.maximize_window()
df = pd.DataFrame()

### Get NBA Team Elo

In [4]:
def get_elo(year):
    url = f'https://projects.fivethirtyeight.com/{year}-nba-predictions'
    browser.get(url)
    
    body = browser.find_element_by_tag_name('tbody')
    teams = body.find_elements_by_tag_name('tr')
    team_names = [t.get_attribute("data-team") for t in teams]
    
    #fill-in missing character
    for i, t in enumerate(teams):
        if len(team_names[i]) < 3:
            # missing character
            c = t.find_element_by_tag_name('a').text[0]
            team_names[i] += c
            
    with tqdm(teams) as pbar:
        pbar.set_description("Get Elo")        
        team_elos = [t.find_element_by_tag_name('td').text
                                         for t in pbar]
            
    with open('preseason_elo.csv', 'w') as f:
        f.write('Name, Elo\n')
        for n, e in zip(team_names, team_elos):
            f.write(n + ', ' + e + '\n')

### Get statistics of all NBA teams from last season

In [7]:
def get_old_stat(year):
    url = f'https://www.basketball-reference.com/leagues/NBA_{year - 1}.html'
    page = requests.get(url)

    soup = BeautifulSoup(page.content, 'html.parser')

    tbl = soup.find("table",{"id":"per_game-team"})

    df = pd.read_html(str(tbl))[0]
    df['Team'] = df['Team'].str.replace('*', '', regex=False)
    df.drop(df.tail(1).index, inplace=True)
    df.to_csv('preseason_data.csv', header=True, index=False)
    
    with open('full_name.txt', 'w') as f:
        for i, link in enumerate(tbl.find_all('a')):
            short = link.get('href').split('/')[2]
            f.write(short + ', ' + df['Team'][i] + '\n')

### Get colum descriptions

In [6]:
def get_description(year):
    url = f'https://www.basketball-reference.com/leagues/NBA_{year - 1}.html'
    browser.get(url)
    
    table = browser.find_element_by_id('per_game-team')
    header = table.find_element_by_tag_name('thead')
    col_tags = header.find_elements_by_tag_name('th')
    date = ['Date']
    
    # create cols
    sign = ['H_', 'A_']
    cols = [c.text for c in col_tags]
    tmp = [[sign[i] + c for c in cols] for i in range(2)]
    col_table = date + tmp[0] + tmp[1]
    
    # create descriptions
    sign_description = ['Home ', 'Away ']
    with tqdm(col_tags) as pbar:
        pbar.set_description("Get Description")
        desc = [c.get_attribute("data-tip") 
                                    for c in pbar]
    # Team description missing
    desc[1] = 'Team'
    
    tmp = [[sign_description[i] + d for d in desc] for i in range(2)]
    description = date + tmp[0] + tmp[1]
    
    with open('raw_description.txt', 'w') as f:
        for c, d in zip(col_table, description):
            f.write(c + ' : ' + d + '\n')

### Get URL to every match in a month

In [7]:
def get_html(url):
    browser.get(url)
    
    btns = browser.find_elements_by_xpath('//*[@data-stat="box_score_text"]')
    btns = [b for b in btns if b.text != ' ']
    links = [b.find_elements_by_xpath('.//*')[0].get_attribute('href') for b in btns]
    
    html_text = browser.page_source
    
    tree = BeautifulSoup(html_text, 'html.parser')
    return links, tree

### Get column names

In [8]:
def get_info(url):
    browser.get(url)

    team = browser.find_element_by_id('line_score')\
                .find_element_by_tag_name('a').text

    table = browser.find_element_by_id(f'box-{team}-game-basic')
    
    header = table.find_element_by_tag_name('thead')
    col_tags = header.find_elements_by_tag_name('th')
    date = ['Date']
    
    # create cols
    sign = ['H_', 'A_']
    cols = ['Team'] + [c.text for c in col_tags][3:]
    tmp = [[sign[i] + c for c in cols] for i in range(2)]
    col_table = date + tmp[0] + tmp[1]
            
    return col_table

### Get Data from every match

In [9]:
def get_data(url):
    browser.get(url)
    
    datetime = browser.find_element_by_class_name('scorebox_meta')\
                        .find_element_by_tag_name('div').text
    time, date = datetime.split(', ', 1)
    
    
    tmp = browser.find_element_by_id('line_score')\
                .find_elements_by_tag_name('a')
    teams = [t.text for t in tmp][::-1] # reverse() // home first

    tables = [browser.find_element_by_id(f'box-{t}-game-basic') 
                                for t in teams]
    data_table = [date]
    for i, t in enumerate(tables):
        footer = t.find_element_by_tag_name('tfoot')
        data_tags = footer.find_elements_by_tag_name('td')
        data = [teams[i]] + [d.text for d in data_tags]
        data_table += data
    
    
    return data_table

### Main 

In [10]:
main_url = 'https://www.basketball-reference.com'
cur_year = 2021
n = 3
years = [cur_year - i for i in range(n - 1, -1, -1)]

cols = []

get_full_name()
get_elo(years[0])
get_old_stat(years[0])
get_description(years[0])

for year in years:
    print(year)
    year_url = f'/leagues/NBA_{year}_games.html'
    urls, tree = get_html(main_url + year_url)

    filter = tree.find('div', class_=['filter'])

    for i, tag in enumerate(filter.find_all('a')):
        if i: # first link same with main page // no need to get html
            link = main_url + tag['href']
            urls, tree = get_html(link)
            
        with tqdm(urls) as pbar:
            pbar.set_description("Processing %s" % tag.text)
            for link in pbar:
                if not cols:
                    cols = get_info(link)
                    df = pd.DataFrame(columns=cols)

                df = df.append(pd.DataFrame([get_data(link)], columns=cols),
                               ignore_index = True)

    break

Get Description: 100%|████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 167.19it/s]


2019


Processing June: 100%|███████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.85s/it]


In [11]:
df.shape

(1312, 43)

In [12]:
df = df.iloc[::-1] # reverse dataframe // from lastest -> oldest
df.head()
df.to_csv('raw_data.csv', index=False)

$R_i$: Elo of a team in $i^{th}$ match

$R_{i + 1} = k * (S_{team} - E_{team} + R_i)$

$S_{team} = 1$: win || $0$: lose

$E_{team} = \frac{1}{1 + 10^{\frac{oppElo - teamElo}{400}}}$

$k = 20 \frac{(MOV_{winner} + 3)^{0.8}}{7.5 + 0.006(\text{Elo_diff}_{winner})}$

$MOV_{winner} = winner_{score} - loser_{score}$

$\text{Elo_diff}_{winner} = winner_{elo} - loser_{elo}$

In [13]:
def K(MOV, elo_diff):
    f = 1 if MOV > 0 else -1 #factor

    k = 20 * (f * MOV + 3) ** (0.8) / (7.5 + 0.006 * (f * elo_diff))

    return k, -k # win: +elo, lose: -elo

In [14]:
def S(MOV):
    S_home, S_away = 0, 0
    if MOV > 0:
        S_home = 1
    elif MOV < 0:
        S_away = 1
    else:
        S_home, S_away = .5, .5
    return S_home, S_away

In [15]:
def elo_prediction(home_elo,away_elo):
    E_home = 1./(1 + 10 ** ((away_elo - home_elo) / (400.)))
    return E_home

In [16]:
def score_prediction(home_elo,away_elo):
    return (home_elo - away_elo)/28.

In [17]:
def elo_update(home_score, away_score, home_elo, away_elo):
    home_advan = 100. #home advantage
    home_elo += home_advan
    E_home = elo_prediction(home_elo, away_elo)
    E_away = 1 - E_home
    elo_diff = home_elo - away_elo
    MOV = home_score - away_score # margin of victory
    
    S_home, S_away = S(home_score,  away_score)
    
    K_home, K_away = K(MOV, elo_diff)
        
    return K_home * (S_home - E_home), K_away * (S_away - E_away)